In [1]:
import sqlite3
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import numpy as np
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

import pickle
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
import os

In [2]:
conn = sqlite3.connect(r"C:\Users\reekithak\notebooks\AzureML\Drug_discovery Files\drugs.db")

There are 65 graph sets 

In [3]:
#generating train_set lists

In [4]:
xl_ = []
for x in range(0,65):
    xl_.append("train_set{}".format(x))

In [5]:
def fit_model(clf, x_t, y_t, x_v=None, y_v=None):
    clf.fit(x_t, y_t)
    print("Training Score", clf.score(x_t, y_t))

    if x_v is not None:
        y_pred = clf.predict(x_v)
        print("Validation Score", clf.score(x_v, y_v))
        print("F1 Score", metrics.f1_score(y_pred, y_v, average="micro"))
        # print("ROC AUC Score", metrics.roc_auc_score(y_pred, y_v))
        print("Recall Score", metrics.recall_score(y_pred, y_v, average="micro"))
    return clf

In [18]:
#xl_ = ["train_set6","train_set7","train_set8"]

In [16]:
main_dir = "C:/Users/reekithak/notebooks/AzureML/Drug_discovery Files/Models/"

In [17]:
for set_ in xl_:
    df = pd.read_sql("SELECT * from {}".format(set_),conn)
    cids = df['cid'].unique() 
    print("Fetching Completed","",set_)
    unique_targets = set(",".join(df['target'].values.flatten().tolist()).split(","))
    #Creating data flow
    id_dict = {v:k for k,v in dict(enumerate(unique_targets)).items()}
    X = []
    y = []
    for cid in cids:
        indxs = np.where(df['cid']==cid)[0]
        X.append(df.iloc[indxs,1:-1].values)

        yi = np.zeros(len(id_dict))
        targets = ",".join(df.iloc[indxs,-1].values.tolist()).split(",")
        targets = [id_dict[i] for i in targets if i!=""]
        yi[targets] = 1
        y.append(yi)
    X = np.array(X).reshape(-1,32)
    y = np.array(y)
    scaler = StandardScaler()
    scaler.fit(X)
    print("Data Flow Completed")
    
    
    
    print(set_)
    path = os.path.join(main_dir,set_)
    os.mkdir(path,mode=0o666)
    if df.shape[0]<100000:
        x_t , x_v , y_t , y_v = train_test_split(X,y,test_size = 0.1,random_state=42)
    else:
        x_t , x_v , y_t , y_v = train_test_split(X,y,test_size = 0.01,random_state=42)
    print("Model Building Starting")
    
    #Logistic regression
    try:
        
        lgr = OneVsRestClassifier(LogisticRegression())
        lgr = lgr.fit(x_t, y_t)
        filename = "/Regressor_model.sav"
        filename = path+ filename
        pickle.dump(lgr,open(filename,'wb'))
    except:
        pass
    
    
    #SVC
    try:
        svc = OneVsRestClassifier(SVC())
        svc = svc.fit(x_t,y_t)
        filename = "/SVC_model.sav"
        filename = path+ filename
        pickle.dump(svc,open(filename,'wb'))
    except:
        pass
    
    #KNN
    try:
        
        knn = OneVsRestClassifier(KNeighborsClassifier())
        knn = knn.fit(x_t,y_t)
        filename = "/KNN_model.sav"
        filename = path+ filename
        pickle.dump(knn,open(filename,'wb'))
    except:
        pass
    
    print("Model Building Done")
    print("******************************************************************************************")
    print("******************************************************************************************")
    print("******************************************************************************************")

Fetching Completed  train_set6
Data Flow Completed
train_set6
Model Building Starting
Model Building Done
******************************************************************************************
******************************************************************************************
******************************************************************************************
Fetching Completed  train_set7
Data Flow Completed
train_set7
Model Building Starting
Model Building Done
******************************************************************************************
******************************************************************************************
******************************************************************************************
Fetching Completed  train_set8
Data Flow Completed
train_set8
Model Building Starting
Model Building Done
******************************************************************************************
*********************************************

Time Consuming

In [83]:
"""rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf = fit_model(rf,x_t, y_t, x_v, y_v)
xgb = XGBClassifier()
xgb = fit_model(xgb,x_t, y_t, x_v, y_v)
"""

'rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)\nrf = fit_model(rf,x_t, y_t, x_v, y_v)\nxgb = XGBClassifier()\nxgb = fit_model(xgb,x_t, y_t, x_v, y_v)\n'